In [ ]:

import pandas as pd
import csv
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist
from matplotlib import pyplot as plt
from keras.utils import plot_model
from keras.utils.vis_utils import model_to_dot
from keras.models import load_model
from keras.models import model_from_json
from keras import backend as K
K.set_image_dim_ordering('tf')
np.random.seed(123)  # for reproducibility

########## TRAINING DATA ##########

print('loading data...')
# load dataset
#dataframe = pd.read_csv('data/train.xlsx', header=None, nrows=1000)
# read them in
excel = pd.ExcelFile('data/validate.xlsx')
    
# turn them into dataframes
dataframe = excel.parse(excel.sheet_names[0], header=None,index_col=None)

print('getting values...')
dataset = dataframe.values
Y = np.array(dataset[:,:-1].astype(float))
X = np.array(dataset[:,-1])

#one-hot encoding
Y = Y.reshape(-1)
s = pd.Series(Y)
Y = np.array(pd.get_dummies(s))

X = [list(map(int, X[i].split())) for i in range(len(X))]
X = np.array(X)

# multidim array
X = np.array([np.split(x_i,48) for x_i in X])

########## TEST DATA ##########

print('loading data...')
# load dataset
#dataframe = pd.read_csv('data/test.xlsx', header=0)
# read them in
excel = pd.ExcelFile('data/test.xlsx')
    
# turn them into dataframes
dataframe = excel.parse(excel.sheet_names[0], header=None,index_col=None)
print('getting values...')
dataset = dataframe.values
Y_t = np.array(dataset[:,:-1].astype(float))
X_t = np.array(dataset[:,-1])

#one-hot encoding
Y_t = Y_t.reshape(-1)
s = pd.Series(Y_t)
Y_t = np.array(pd.get_dummies(s))

X_t = [list(map(int, X_t[i].split())) for i in range(len(X_t))]
X_t = np.array(X_t)

# multidim array
X_t = np.array([np.split(x_i,48) for x_i in X_t])

########## DATA PREPROCESSING ##########

X_t = X_t.reshape(X_t.shape[0], 48, 48, 1)
X = X.reshape(X.shape[0], 48, 48, 1)

X = X.astype('float32')
X_t = X_t.astype('float32')
X /= 255
X_t /= 255

########## LOSS FUNCTION ###########

def custom_objective(y_true, y_pred):
    '''Just another crossentropy'''
    y_pred = T.clip(y_pred, epsilon, 1.0 - epsilon)
    y_pred /= y_pred.sum(axis=-1, keepdims=True)
    cce = T.nnet.categorical_crossentropy(y_pred, y_true)
    return cce

# Add it to optimiser like this
# model.compile(loss= custom_objective,
#              optimizer=keras.optimizers.Adam(lr=1e-7),
#              metrics=['accuracy'])

########## KERAS ML MODEL ##########

model = Sequential()
model.add(Convolution2D(64, (3, 3), activation='relu', input_shape=(48,48, 1), padding='same'))
#model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

model.add(Convolution2D(128, (3, 3), activation='relu', padding='valid'))
#model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

model.add(Convolution2D(256, (3, 3), activation='relu', padding='valid'))
#model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

model.add(Convolution2D(512, (3, 3), activation='relu', padding='valid'))
#model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

model.add(Convolution2D(512, (3, 3), activation='relu', padding='valid'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

# print (model.output_shape)

model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(lr=1e-7, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
              metrics=['accuracy'])

history = model.fit(X, Y, validation_split=0.33, batch_size=48, epochs=3, verbose=1)
scores = model.evaluate(X_t, Y_t, verbose=0)

print(model.summary())
print(scores)

# output model visualization to file
plot_model(model, to_file='out/model.png')

# ########## SAVE MODEL ##########

print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("out/model.h5")
print("Saved model to disk")

########## LOAD MODEL ##########

# json_file = open('dillion_model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights("dillion_model.h5")
# print("Loaded model from disk")
#
# # evaluate loaded model on test data
# loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# score = loaded_model.evaluate(X_t, Y_t, verbose=0)
# print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))
#
# prediction = loaded_model.predict(X, batch_size=1, verbose=0)
# print("prediction of emotion", prediction)
#
# predict_1 = prediction[0]
# emotion_arr = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]
#
# def emotion(arr):
#     index = 0
#     highest_num = 0
#     for i in range(7):
#         if (arr[i] > highest_num):
#             index = i
#             highest_num = arr[i]
#     return emotion_arr[index]
#
# for p in prediction:
#     print(emotion(p), p)

########## SHOW GRAPH OF RESULTS ##########

# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


loading data...
getting values...
loading data...
getting values...
Train on 2404 samples, validate on 1185 samples
Epoch 1/3
1680/2404 [===================>..........] - ETA: 5:55 - loss: 1.9449 - acc: 0.1482